## Model Analysis Script

#### General input:

In [1]:
from IPython.display import display, Image
from rmgpy.molecule import Molecule
import numpy as np
import sys
import os
sys.path.append(os.getcwd())
import SuppFunc as sf
%matplotlib notebook

run_name = 'testNewMaster'
Fuel = 'NCC' # SMILES. this is only used for drawing the molecule
topSA = 10   # top SA entries to plot; change to 0 to skip SA
minSAtime = 0 # time window to display top SA, sec
maxSAtime = 0.000025
SA_species_list = ['OH','NH2'] # make sure this matches the input file! (unless topSA is 0)
ROPtime = 350e-6 # time for calculating flux in ROP diagram
element = 'C' # element to track in flux analysis

Fuel_Molecule = Molecule().fromSMILES(Fuel)
print("The fuel molecule is:");display(Fuel_Molecule)

The fuel molecule is:


#### RMG's input file:
Caution !!! Running this will also delete all files under the run folder !!!

In [7]:
inputFile = '''database(
    thermoLibraries=['BurkeH2O2','thermo_DFT_CCSDTF12_BAC','NitrogenCurran','primaryThermoLibrary','DFT_QCI_thermo','FFCM1(-)','CHN','GRI-Mech3.0-N'],
    reactionLibraries=['Ethylamine','FFCM1(-)','Nitrogen_Dean_and_Bozzelli','Nitrogen_Glarborg_Zhang_et_al'],
    seedMechanisms=['BurkeH2O2inArHe'],kineticsDepositories=['training'],kineticsFamilies='default',kineticsEstimator='rate rules')
species(label='NCC', reactive=True, structure=SMILES("NCC"))
species(label='O2', reactive=True, structure=SMILES("[O][O]"))
species(label='Ar', reactive=False, structure=adjacencyList("""1 Ar u0 p4 c0"""))
species(label='OH', reactive=True, structure=SMILES("[OH]"))
species(label='NH2', reactive=True, structure=SMILES("[NH2]"))
#EA-Pyr(NH2)
simpleReactor(temperature=(1428,'K'), pressure=(1.21,'atm'),
    initialMoleFractions={"NCC": 0.002, "Ar": 0.998},
    terminationTime=(0.0005,'s'),
    sensitivity=['NH2'],sensitivityThreshold=0.01
)
#EA-Oxy(NH2)
simpleReactor(temperature=(1441,'K'), pressure=(2.09,'atm'),
    initialMoleFractions={"NCC": 0.002, "O2": 0.008, "Ar": 0.99},
    terminationTime=(0.0005,'s'),
    sensitivity=['NH2','OH'],sensitivityThreshold=0.01
)
#EA-Oxy(OH)
simpleReactor(temperature=(1399,'K'), pressure=(1.93,'atm'),
    initialMoleFractions={"NCC": 0.0005, "O2": 0.002, "Ar": 0.9975},
    terminationTime=(0.0025,'s'),
#    sensitivity=['OH'],sensitivityThreshold=0.01
)
simulator(atol=1e-16, rtol=1e-8, sens_atol=1e-6, sens_rtol=1e-4)
model(
    toleranceKeepInEdge=0,
    toleranceMoveToCore=0.02,
    toleranceInterruptSimulation=0.02,
    maximumEdgeSpecies=300000)
pressureDependence(
    method='modified strong collision',
    maximumGrainSize=(0.5,'kcal/mol'),
    minimumNumberOfGrains=250,
    temperatures=(298,2500,'K',10),
    pressures=(0.1,10,'bar',10),
    interpolation=('Chebyshev', 6, 4),
    maximumAtoms=16)
options(units='si', generateOutputHTML=True, generatePlots=False, saveEdgeSpecies=True,
    saveSimulationProfiles=False, saveRestartPeriod=None)
generatedSpeciesConstraints(
    allowed=['input species','seed mechanisms','reaction libraries'],
    maximumCarbonAtoms=3,
    maximumOxygenAtoms=2,
    maximumNitrogenAtoms=2,
    maximumSiliconAtoms=0,
    maximumSulfurAtoms=0,
    maximumHeavyAtoms=3,
    maximumRadicalElectrons=3,
    allowSingletO2=False,
)
'''
sf.createInput(run_name,inputFile)

Crerated RMG input file: testNewMaster/input.py


#### Run RMG:

In [ ]:
#os.system('python %RMG%/rmg.py '+run_name+'/input.py')    # run RMG with the above input file
os.system('python $rmg/rmg.py '+run_name+'/input.py')
sf.RMGlog(run_name)

RMG Simulation Completed. Summary of log file:

MODEL GENERATION COMPLETED

The final model core has 80 species and 1884 reactions

The final model edge has 291 species and 9145 reactions



#### Analyze model in Cantera:

In [ ]:
from rmgpy.chemkin import *
from rmgpy.tools.canteraModel import *
from rmgpy.species import Species
import time

path = run_name+"/chemkin/"
speciesList, reactionList = loadChemkinFile(path+'chem_annotated.inp',
                                            path+'species_dictionary.txt',)
                                            #path+'tran.dat')

EA_Species=Species().fromSMILES('NCC')
O2_Species=Species().fromSMILES('[O][O]')
Ar_Species=Species().fromAdjacencyList("""1 Ar u0 p4 c0""")
NH2_Species=Species().fromSMILES('[NH2]')
OH_Species=Species().fromSMILES('[OH]')
species_dict = getRMGSpeciesFromUserSpecies([EA_Species,O2_Species,Ar_Species,NH2_Species,OH_Species], speciesList)

reactorTypeList = ['IdealGasConstPressureReactor']

reactionTimeList0 = ([0.5], 'ms')
molFracList0=[{species_dict[EA_Species]:0.002, species_dict[O2_Species]:0.0, species_dict[Ar_Species]:0.998}]
T0 = ([1428],'K'); P0 = ([1.21],'atm')

reactionTimeList1 = ([0.4], 'ms')
molFracList1=[{species_dict[EA_Species]:0.002, species_dict[O2_Species]:0.008, species_dict[Ar_Species]:0.99}]
T1 = ([1441],'K'); P1 = ([2.09],'atm')

reactionTimeList2 = ([2.5], 'ms')
molFracList2=[{species_dict[EA_Species]:0.0005, species_dict[O2_Species]:0.002, species_dict[Ar_Species]:0.9975}]
T2 = ([1399],'K'); P2 = ([1.93],'atm')

print("Working on it...")
job0 = Cantera(speciesList=speciesList, reactionList=reactionList, outputDirectory=path+'/NH2_pyr')
job0.loadChemkinModel(path+'chem_annotated.inp',transportFile=path+'tran.dat')
job0.generateConditions(reactorTypeList, reactionTimeList0, molFracList0, T0, P0)

job1 = Cantera(speciesList=speciesList, reactionList=reactionList, outputDirectory=path+'/NH2_oxi')
job1.loadChemkinModel(path+'chem_annotated.inp',transportFile=path+'tran.dat')
job1.generateConditions(reactorTypeList, reactionTimeList1, molFracList1, T1, P1)

job2 = Cantera(speciesList=speciesList, reactionList=reactionList, outputDirectory=path+'/OH_oxi')
job2.loadChemkinModel(path+'chem_annotated.inp',transportFile=path+'tran.dat')
job2.generateConditions(reactorTypeList, reactionTimeList2, molFracList2, T2, P2)

PyrNH2_data = job0.simulate()
OxyNH2_data = job1.simulate()
OxyOH_data =  job2.simulate()
print("Done.")

Working on it...
Wrote CTI mechanism file to 'testNewMaster/chemkin//NH2_pyr/chem_annotated.cti'.
Mechanism contains 80 species and 1891 reactions.
Wrote CTI mechanism file to 'testNewMaster/chemkin//NH2_oxi/chem_annotated.cti'.
Mechanism contains 80 species and 1891 reactions.
Wrote CTI mechanism file to 'testNewMaster/chemkin//OH_oxi/chem_annotated.cti'.
Mechanism contains 80 species and 1891 reactions.


#### Plot speciation

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from rmgpy.tools import plot as rmg_plot
from operator import itemgetter
from rmgpy.tools.sensitivity import runSensitivity
%matplotlib notebook

######### Speciation:
for spc in xrange(len(OxyNH2_data[0][1][:])):       # find index for NH2
    if OxyNH2_data[0][1][spc].label == str(species_dict[NH2_Species]):
        NH2_idx = spc
for spc in xrange(len(OxyOH_data[0][1][:])):       # find index for NH2
    if OxyOH_data[0][1][spc].label == str(species_dict[OH_Species]):
        OH_idx = spc

Hanson = pd.read_csv('experimental/Hanson.csv')       # Load Hanson's data
time0 = PyrNH2_data[0][0].data
time1 = OxyNH2_data[0][0].data
time2 = OxyOH_data[0][0].data

fig = plt.figure(figsize=(9,9))

plt.subplot(2,2,1)
plt.plot(Hanson['PyrNH2 Time (ms)'],Hanson['PyrNH2 NH2 (ppm)'],'k-')
plt.plot(time0*1e6, PyrNH2_data[0][1][NH2_idx].data*1e6,'b--',label='NH2')
plt.xlabel(r'Time ($\mu$s)')
plt.ylabel(r'NH$_2$ (ppm)')
plt.title(r'Pyrolysis NH$_2$ profile ({0})'.format(run_name))
plt.xlim(0, reactionTimeList0[0][0] * 1e3)
plt.ylim(0, 140)

plt.subplot(2,2,3)
plt.plot(Hanson['OxyNH2 Time (ms)'],Hanson['OxyNH2 NH2 (ppm)'],'k-')
plt.plot(time1*1e6, OxyNH2_data[0][1][NH2_idx].data*1e6,'b--',label='NH2')
plt.xlabel(r'Time ($\mu$s)')
plt.ylabel(r'NH$_2$ (ppm)')
plt.title(r'NH$_2$ profile ({0})'.format(run_name))
plt.xlim(0, reactionTimeList1[0][0] * 1e3)
plt.ylim(0, 160)

plt.subplot(2,2,4)
plt.plot(Hanson['OxyOH Time (ms)'],Hanson['OxyOH OH (ppm)'],'k-')
plt.plot(time2*1e6, OxyOH_data[0][1][OH_idx].data*1e6,'b--',label='OH')
plt.xlabel(r'Time ($\mu$s)')
plt.ylabel('OH (ppm)')
plt.title('OH profile ({0})'.format(run_name))
plt.xlim(0, reactionTimeList2[0][0] * 1e3)

plt.tight_layout()
plt.show()

sf.saveFig(plt,run_name,subname='speciation'); print('Figure saved')

#### Plot sensitivity analysis

In [ ]:
#SAdata format: SAdata[species][0 for time, 1 for data]; add .data or .label if it's time; else add [reaction] and the .data or .label; eg: SAdata[0][0].data gives times for species 0, SAdata[0][1][2].data gives data of reaction 2 for species 0

print("Plotting SA for species {0}, {1} at the selected time range {2}-{3} sec\n".format(
    SA_species_list[0],SA_species_list[1],minSAtime,maxSAtime))
cmap = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#8c564b','#c49c94','#e377c2','#f7b6d2','#7f7f7f','#c7c7c7','#bcbd22','#dbdb8d','#17becf','#9edae5']

if topSA:
    files = os.listdir(run_name+'/solver')
    SA_files = filter(lambda x: ('sensitivity' in x) and ('.csv' in x),files)
    fig = plt.figure(figsize=(9,15*0.5*len(SA_files)))
   # plt.style.use('seaborn-darkgrid')
    SAdata = [] # initialize SAdata where time+SA is saved
    minSAidx = []; maxSAidx = [] # a list of indices for minimum value and maximum value for SA sort and plot
    G = np.zeros(len(SA_files)) # marke the point where SA data strats representing G instead of k sensitivities
    for SAidx in xrange(len(SA_files)):
        SAdata.append(rmg_plot.parseCSVData(run_name+'/solver/'+SA_files[SAidx])) # structure of SAdata is an array of time;data
        for i in xrange(len(SAdata[SAidx][1][:])):  # SA_idx refers to the last appended entry, 1 refers to data (as opposed to 0 for time), : refers to all
            if "G" in SAdata[SAidx][1][i].label:
                if not G[SAidx]: G[SAidx] = i # save i only if this is the first occurence
                SAdata[SAidx][1][i].label = SAdata[SAidx][1][i].label.split('G')[1][1:-1]  # Just prettify the headers accordingly
            else:
                SAdata[SAidx][1][i].label = SAdata[SAidx][1][i].label.split(' ')[1]        # Just prettify the headers accordingly
        minSAidx.append(sf.findClosest(minSAtime,SAdata[SAidx][0].data))
        maxSAidx.append(sf.findClosest(maxSAtime,SAdata[SAidx][0].data))
        
        rank = []; data = []; label = []
        for n in xrange(int(G[SAidx])):   # run only for k now (up to the G marker)
            rank.append(max(abs(SAdata[SAidx][1][n].data[minSAidx[SAidx]:maxSAidx[SAidx]]))) # list of max SA range for each rxn
        num = np.linspace(0,len(rank)-1,len(rank)) # generate a 0,1,2,3... list
        num = zip(rank,num)
        num = sorted(num, key=itemgetter(0),reverse=True)
        for i in xrange(min(topSA, G[SAidx])):    # G[SAidx] is there in case there are less than topSA sensitive reactions
            data.append(SAdata[SAidx][1][int(num[i][1])].data[:])
            label.append(SAdata[SAidx][1][int(num[i][1])].label[:])
        for i in xrange(min(topSA, G[SAidx])):
            SAdata[SAidx][1][i].data  = data[i]   # overwriting some of the values in SAdata, so be sure to only call topSA items from there
            SAdata[SAidx][1][i].label = label[i]  # a possible issue is that if there are less than topSA relevant rxns, we won;t know about it when plotting...
        
        rank = []; data = []; label = []
        for n in xrange(len(SAdata)-int(G[SAidx])):   # run only for G now (from the G marker)
            rank.append(max(abs(SAdata[SAidx][1][n+G[SAidx]].data[minSAidx[SAidx]:maxSAidx[SAidx]]))) # list of max SA range for each G
        num = np.linspace(0,len(rank)-1,len(rank)) # generate a 0,1,2,3... list
        num = zip(rank,num)
        num = sorted(num, key=itemgetter(0),reverse=True)
        for i in xrange(min(topSA, len(SAdata)-int(G[SAidx]))):
            data.append(SAdata[SAidx][1][int(num[i+G[SAidx]][1])].data[:])
            label.append(SAdata[SAidx][1][int(num[i+G[SAidx]][1])].label[:])
        for i in xrange(min(topSA, len(SAdata)-int(G[SAidx]))):
            SAdata[SAidx][1][i+G[SAidx]].data  = data[i]
            SAdata[SAidx][1][i+G[SAidx]].label = label[i]

        SAk = plt.subplot(2*len(SA_files),2,SAidx*4+1)
        ymin = ymax = 0
        for n in xrange(min(topSA, G[SAidx])):
            SAk.plot(SAdata[SAidx][0].data, SAdata[SAidx][1][n].data, color=cmap[n], label=SAdata[SAidx][1][n].label)
            if min(SAdata[SAidx][1][n].data[minSAidx[SAidx]:maxSAidx[SAidx]]) < ymin:
                ymin = min(SAdata[SAidx][1][n].data[minSAidx[SAidx]:maxSAidx[SAidx]])
            if max(SAdata[SAidx][1][n].data[minSAidx[SAidx]:maxSAidx[SAidx]]) > ymax:
                ymax = max(SAdata[SAidx][1][n].data[minSAidx[SAidx]:maxSAidx[SAidx]])
        plt.xlim(minSAtime, min(maxSAtime,SAdata[SAidx][0].data[-1]))
        plt.ylim(ymin,ymax)
        plt.xlabel('Time (s)')
        plt.ylabel(r'Normalized sensetivity coefficient, $\frac{\partial\:\ln{[X]}}{\partial\:\ln{k}}$')
        plt.title('Kinetics SA for {0}'.format(SA_species_list[SAidx]))
        plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
        
        SAg = plt.subplot(2*len(SA_files),2,SAidx*4+2)
        ymin = ymax = 0
        for n in xrange(min(topSA, G[SAidx])):
            SAg.plot(SAdata[SAidx][0].data, SAdata[SAidx][1][n+int(G[SAidx])].data, color=cmap[n], label=SAdata[SAidx][1][n+int(G[SAidx])].label)
            if min(SAdata[SAidx][1][n+int(G[SAidx])].data[minSAidx[SAidx]:maxSAidx[SAidx]]) < ymin:
                ymin = min(SAdata[SAidx][1][n+int(G[SAidx])].data[minSAidx[SAidx]:maxSAidx[SAidx]])
            if max(SAdata[SAidx][1][n+int(G[SAidx])].data[minSAidx[SAidx]:maxSAidx[SAidx]]) > ymax:
                ymax = max(SAdata[SAidx][1][n+int(G[SAidx])].data[minSAidx[SAidx]:maxSAidx[SAidx]])
        plt.xlim(minSAtime, min(maxSAtime,SAdata[SAidx][0].data[-1]))
        plt.ylim(ymin,ymax)
        plt.xlabel('Time (s)')
        plt.ylabel(r'Sensetivity coefficient, $\frac{\partial\:\ln{[X]}}{\partial\:G_i}$ $(mol/kcal)$')
        plt.title('Thermo SA for {0}'.format(SA_species_list[SAidx]))
        plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
        
        SAkLeg = plt.subplot(2*len(SA_files),2,SAidx*4+3)
        for n in xrange(topSA):
            SAkLeg.plot(0,0, color=cmap[n], label=SAdata[SAidx][1][n].label)
        SAkLeg.legend(loc='upper center', fancybox=True, shadow=True)
        SAkLeg.spines['bottom'].set_color('white')
        SAkLeg.spines['top'].set_color('white')
        SAkLeg.spines['left'].set_color('white')
        SAkLeg.spines['right'].set_color('white')
        SAkLeg.tick_params(color='white', labelcolor='white')
        
        SAgLeg = plt.subplot(2*len(SA_files),2,SAidx*4+4)
        for n in xrange(topSA):
            SAgLeg.plot(0,0, color=cmap[n], label=SAdata[SAidx][1][n+int(G[SAidx])].label)
        SAgLeg.legend(loc='upper center', fancybox=True, shadow=True)
        SAgLeg.spines['bottom'].set_color('white')
        SAgLeg.spines['top'].set_color('white')
        SAgLeg.spines['left'].set_color('white')
        SAgLeg.spines['right'].set_color('white')
        SAgLeg.tick_params(color='white', labelcolor='white')

    plt.tight_layout()
    plt.show()
    sf.saveFig(fig,run_name,subname='SA'); print('Figure saved')

#### Plot flux diagram

In [ ]:
gas = ct.Solution(run_name + '/cantera/chem.cti')
comp = []
for mol, molfrac in molFracList1[0].iteritems():
    comp.append(str(mol) + ':' + str(molfrac))
gas.TPX = T1[0][0], P1[0][0], str(','.join(comp))
reactor = ct.IdealGasConstPressureReactor(gas)
network = ct.ReactorNet([reactor])
network.advance(ROPtime)
ROP = ct.ReactionPathDiagram(gas, element)

dot_file = run_name+'/AnalysisScriptFigs/rxnpath.dot'
img_file = run_name+'/AnalysisScriptFigs/rxnpath.png'
ROP.title = 'Reaction path diagram following {0} at time = {1} sec'.format(element,ROPtime)
ROP.threshold = 0.00001
ROP.label_threshold = 0.00001
ROP.show_details = True
ROP.write_dot(dot_file)                                               # write dot file
os.system('dot {0} -Tpng -o{1} -Gdpi=300'.format(dot_file, img_file)) # write png file
print('Figure saved')
Image(os.getcwd() + '/' + img_file)

In [ ]:
int(G[SAidx])